<a href="https://colab.research.google.com/github/h777arsh/PyTorch-Tutorial/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
#read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [7]:
len(words)

32033

In [8]:
#build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = { s: i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [63]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], [] # X = input, Y = Label for each input of X
for w in words:

  # print(w)
  context = [0] * block_size
  for ch in w + '.':
    # print('ch: '+ch)
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    # print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
    # print(context)

X = torch.tensor(X)
Y = torch.tensor(Y)

In [64]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [11]:
# #one hot example
# # Suppose we have the following indices
# indices = torch.tensor([0, 1, 2, 3])

# # The number of categories (classes) we have
# num_classes = 4

# # Convert indices to one-hot encoded format
# one_hot_encoded = F.one_hot(indices, num_classes)

# print(one_hot_encoded)

In [67]:
C = torch.randn((27,2))

In [68]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [69]:
W1 = torch.rand((6, 100))
b1 = torch.rand(100)
h = emb.view(-1, 6) @ W1 + b1
h

tensor([[-0.4013,  0.4344,  0.3002,  ...,  0.1768, -0.2630, -0.6592],
        [ 0.0879,  1.9019,  0.0132,  ...,  0.5753,  0.6087, -0.6134],
        [-2.5585, -0.3644, -0.1615,  ..., -2.2408, -0.9084, -2.4027],
        ...,
        [ 0.1571,  0.5353, -0.1902,  ..., -0.0435, -0.5565,  0.2832],
        [-0.2825,  0.4005, -0.4507,  ..., -0.0599, -0.2228, -0.1499],
        [-0.2618,  0.4448, -0.1265,  ..., -0.0730, -0.3113,  0.0131]])

In [70]:
h.shape

torch.Size([228146, 100])

In [71]:
(emb.view(-1, 6) @ W1).shape

torch.Size([228146, 100])

In [72]:
b1.shape

torch.Size([100])

In [18]:
# 32, 100
#  1, 100  Broadcasting with right, will add 1 on empty

In [43]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [73]:
logits = h @ W2 + b2

In [74]:
logits.shape

torch.Size([228146, 27])

In [75]:
counts = logits.exp()

In [76]:
prob = counts / counts.sum(1, keepdims = True)

In [77]:
prob.shape

torch.Size([228146, 27])

In [78]:
prob[0].sum()

tensor(1., grad_fn=<SumBackward0>)

In [79]:
# prob[torch.arange(32), Y]

In [52]:
# loss = -prob[torch.arange(32), Y].log().mean()
# loss

tensor(23.8543)

In [29]:
# Now made respectable

In [80]:
X.shape, Y.shape # dataset

(torch.Size([228146, 3]), torch.Size([228146]))

In [81]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator = g)
W1 = torch.randn((6, 100), generator = g)
b1 = torch.randn((100), generator = g)
W2 = torch.randn((100, 27), generator = g)
b2 = torch.randn((27), generator = g)
parameters = [C, W1, b1, W2, b2]


In [82]:
sum(p.nelement() for p in parameters) # number of parameters in total

3481

In [83]:
for p in parameters:
  p.requires_grad = True

In [85]:
for _ in range(10):
  # forward pass
  emb = C[X] # (32, 3, 2)
  h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
  logits = h @ W2 + b2 # (32, 27)
  # counts = logits.exp()
  # prob = counts / counts.sum(1, keepdims = True)
  # loss = - prob[torch.arange(32), Y].log().mean()
  loss = F.cross_entropy(logits, Y)

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  #update
  for p in parameters:
    p.data += -0.1 * p.grad

print(loss.item())

2.9964165687561035
